# Persona to GPT-4 Interface - Google Colab Version

This notebook allows you to input a persona and get responses from GPT-4. This is a standalone version that works in Google Colab without local file dependencies.

## Setup
1. Run the first cell to install dependencies
2. Set your OpenAI API key when prompted
3. Modify the persona and questions in the designated cells
4. Run cells to get GPT-4 responses

In [ ]:
# Install required dependencies
!pip install openai python-dotenv tenacity tqdm

import os
import asyncio
import time
from typing import Dict, Optional, Union, Callable, List, Tuple
import openai
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
import httpx
from tqdm.asyncio import tqdm_asyncio
import getpass

print("Dependencies installed successfully!")

In [ ]:
# Set up OpenAI API key
if not os.getenv('OPENAI_API_KEY'):
    api_key = getpass.getpass("Enter your OpenAI API key: ")
    os.environ['OPENAI_API_KEY'] = api_key
    print("✅ API key set successfully")
else:
    print("✅ API key already set")

In [ ]:
# Configuration settings (from openai_config.yaml)
CONFIG = {
    'model_name': 'gpt-4.1-mini-2025-04-14',
    'temperature': 0.0,
    'max_tokens': 16384,
    'max_retries': 10,
    'system_instruction': """You are an AI assistant. Your task is to answer the 'New Survey Question' as if you are the person described in the 'Persona Profile' (which consists of their past survey responses). 
Adhere to the persona by being consistent with their previous answers and stated characteristics. 
Follow all instructions provided for the new question carefully regarding the format of your answer."""
}

print("Configuration loaded:")
print(f"Model: {CONFIG['model_name']}")
print(f"Temperature: {CONFIG['temperature']}")
print(f"Max tokens: {CONFIG['max_tokens']}")

In [ ]:
# LLM Helper Functions (simplified version)
class LLMConfig:
    def __init__(self, model_name: str, temperature: float = 0.7, max_tokens: Optional[int] = None, 
                 system_instruction: Optional[str] = None, max_retries: int = 10):
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.system_instruction = system_instruction
        self.max_retries = max_retries

@retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=4, max=60),
    retry=retry_if_exception_type((ConnectionError, TimeoutError, openai.APITimeoutError, 
                                   openai.APIConnectionError, openai.RateLimitError, openai.APIError)),
    reraise=True
)
async def get_openai_response(prompt: str, config: LLMConfig) -> Dict[str, Union[str, Dict]]:
    """Get response from OpenAI API with retry logic."""
    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY environment variable not set")
    
    async with httpx.AsyncClient(timeout=1000.0) as client:
        aclient = openai.AsyncOpenAI(api_key=api_key, http_client=client)
        messages = [
            {"role": "system", "content": config.system_instruction}, 
            {"role": "user", "content": prompt}
        ]
        
        try:
            response = await aclient.chat.completions.create(
                model=config.model_name, 
                messages=messages, 
                temperature=config.temperature, 
                max_tokens=config.max_tokens
            )
            
            usage_details = {
                "prompt_token_count": response.usage.prompt_tokens,
                "completion_token_count": response.usage.completion_tokens,
                "total_token_count": response.usage.total_tokens
            }
            
            return {
                "response_text": response.choices[0].message.content, 
                "usage_details": usage_details
            }
        except Exception as e:
            return {"error": f"API call failed: {str(e)}"}

def create_persona_prompt(persona_profile: str, new_question: str) -> str:
    """Create a formatted prompt with persona profile and new question."""
    return f"""Persona Profile:
{persona_profile}

New Survey Question:
{new_question}"""

async def get_gpt_response(persona_profile: str, new_question: str, config: LLMConfig) -> Dict:
    """Get GPT response for a given persona and question."""
    prompt = create_persona_prompt(persona_profile, new_question)
    return await get_openai_response(prompt, config)

# Create LLM configuration
llm_config = LLMConfig(
    model_name=CONFIG['model_name'],
    temperature=CONFIG['temperature'],
    max_tokens=CONFIG['max_tokens'],
    system_instruction=CONFIG['system_instruction'],
    max_retries=CONFIG['max_retries']
)

print("✅ LLM helper functions and configuration ready!")

## Define Your Persona Profile

Modify the cell below to customize your persona's characteristics and previous survey responses.

In [ ]:
# Define your persona profile
persona_profile = """
Age: 35
Gender: Female
Education: Bachelor's degree in Business Administration
Income: $75,000 annually
Location: Urban area, East Coast
Previous survey responses:
- When asked about work-life balance: "I prioritize family time and try to maintain clear boundaries between work and personal life."
- When asked about technology adoption: "I'm generally cautious about new technologies and prefer to wait until they're proven and widely adopted."
- When asked about spending habits: "I'm a careful budgeter who prefers to save money for future goals rather than spend on immediate gratification."
""".strip()

print("Persona profile set:")
print(persona_profile)

## Define Your Survey Question

Enter the new survey question you want the persona to answer.

In [ ]:
# Define your new survey question
new_question = """
How do you feel about remote work opportunities? Please provide your opinion in 2-3 sentences, considering your work-life balance preferences and technology comfort level.
""".strip()

print("New survey question:")
print(new_question)

## Get GPT-4 Response

Run the cell below to get the GPT-4 response as your defined persona.

In [ ]:
# Get the GPT-4 response
response_data = await get_gpt_response(persona_profile, new_question, llm_config)

# Display the results
if "error" in response_data and response_data["error"]:
    print("❌ Error occurred:")
    print(response_data["error"])
else:
    print("🤖 GPT-4 Response as the persona:")
    print("=" * 50)
    print(response_data["response_text"])
    print("=" * 50)
    
    # Display usage statistics
    if "usage_details" in response_data:
        usage = response_data["usage_details"]
        print(f"\n📊 Token usage:")
        print(f"- Prompt tokens: {usage.get('prompt_token_count', 'N/A')}")
        print(f"- Completion tokens: {usage.get('completion_token_count', 'N/A')}")
        print(f"- Total tokens: {usage.get('total_token_count', 'N/A')}")

## Interactive Testing

Use the cell below to test different questions with the same persona. Simply modify the question and re-run the cell.

In [ ]:
# Interactive testing - modify this question and re-run
interactive_question = "What is your opinion on investing in cryptocurrency?"

print(f"🔄 Testing question: {interactive_question}")
print("-" * 50)

response_data = await get_gpt_response(persona_profile, interactive_question, llm_config)

if "error" in response_data and response_data["error"]:
    print("❌ Error occurred:")
    print(response_data["error"])
else:
    print("🤖 Persona Response:")
    print(response_data["response_text"])
    print("-" * 50)
    if "usage_details" in response_data:
        print(f"📊 Tokens used: {response_data['usage_details'].get('total_token_count', 'N/A')}")

## Batch Testing

Test multiple questions at once with the same persona.

In [ ]:
# Define multiple questions to test
test_questions = [
    "How do you prefer to shop for groceries?",
    "What factors influence your choice of vacation destinations?",
    "How do you stay informed about current events?",
    "What is your opinion on electric vehicles?",
    "How do you approach making major financial decisions?"
]

print("🔄 Processing multiple questions...\n")

total_tokens = 0

for i, question in enumerate(test_questions, 1):
    print(f"❓ Question {i}: {question}")
    
    response_data = await get_gpt_response(persona_profile, question, llm_config)
    
    if "error" in response_data and response_data["error"]:
        print(f"❌ Error: {response_data['error']}")
    else:
        print(f"🤖 Response: {response_data['response_text']}")
        if "usage_details" in response_data:
            tokens = response_data['usage_details'].get('total_token_count', 0)
            total_tokens += tokens
            print(f"📊 Tokens: {tokens}")
    
    print("-" * 80)
    print()

print(f"🎯 Total tokens used for all questions: {total_tokens}")

## Custom Persona Creator

Use this section to quickly create and test a new persona.

In [ ]:
# Create a custom persona
custom_persona = """
Age: 28
Gender: Male
Education: Master's degree in Computer Science
Income: $95,000 annually
Location: San Francisco, California
Previous survey responses:
- When asked about work preferences: "I thrive in fast-paced, innovative environments and enjoy tackling complex technical challenges."
- When asked about technology: "I'm an early adopter who loves trying new gadgets and software as soon as they're released."
- When asked about lifestyle: "I value experiences over material possessions and prefer to spend money on travel and learning opportunities."
""".strip()

# Test question for the custom persona
custom_question = "How do you feel about AI tools in the workplace?"

print("🎭 Testing custom persona:")
print(custom_persona)
print(f"\n❓ Question: {custom_question}")
print("-" * 50)

response_data = await get_gpt_response(custom_persona, custom_question, llm_config)

if "error" in response_data and response_data["error"]:
    print(f"❌ Error: {response_data['error']}")
else:
    print(f"🤖 Response: {response_data['response_text']}")